# Exercise 4: Dimensionality Reduction

Para este ejercicio se implementaron algortimos de **PCA** (Principal Component Analysis) y **SVD** (Singular Value Decomposition) con el fin de comprimir imagenes y comparar la "calidad" de compresión de cada metodo.

## Introducción:

Antes de pasar al codigo, explicaremos un poco funciona cada uno de los metodos y sus ventajas:

En ambos casos, partimos de una mtriz $X \in \mathbb{R}^{n \times d}$ donde $n$ es el número de observaciones y $d$ es la dimensión.

### Principal Component Analysis:

Para **PCA**, sea $X$ una matriz de $n \times d$, buscamos centrar los datos de $X$ al rededor de la media de cada columna como $$X_{centrado} = X - \mu$$ Entonces, podemos obtener la matriz de covarianza de $X_{centrado}$ como: $$ C = \frac{1}{n-1} X_{centrado}^T X_{centrado} $$

Donde $Cv_i = \lambda _i v_i$ tal que $\lambda_1$ y $v_i$ son los igenvalores y eigenvectores de $C$ respectivamente.<br><br>

Entonces, para hacer la reducción de dimensionalidad, seleccionamos los $k$ eigenvalores asociados a los eigenvectores con mayor varianza. Entonces, podemos recomponer a $X$ como una proyección de ma matriz original de la forma: $$X_{reducido} = X_{centrado}V_K$$

Donde $V_k$ es una matriz de proyección



### Singular Value Decomposition

En **SVD**, sea $X$ una matriz de $n \times d$, entonces podemos recomponer a la matriz $X$ de la forma: $$ X = UDV^T$$ Donde $U$ y $V$ son matrices ortogonales de $n \times d$ y $d \times d$ tal que las columnas de $U$ son el espacio de columnas de $X$ y las columnas de $V$ son el espacio de filas de $X$. $D$ es una matriz de $d \times d$ con entradas $d_1  
≥ d_2 ≥ ... ≥ 0$ llamadas los valores singulares de $X$. <br><br>

Para la reducción de dimensionalidad, seleccionamos los primeros $k$ valores singulares $\phi_1, \phi_2, ... \phi_k$ y sus vectores correspondentes, con lo que podemos aproximar a la matriz $X$ como: $$ X ≈ U_k D_k V_k^T$$ Donde $U$, $D$ y $V$ son matrices ortogonales, tal que los valores singulares $\phi_i$ determina la importancia de cada dimensión.

En resumen **PCA** calcula los autovectores de la matriz de covarianza $X^TX$ para proyectar los datos en un espacio de menor dimensión y 
**SVD** descompone la matriz $X$ en $ UDV^T$ lo que permite reducir la dimensión en funcion de los $k$ valores singulares más importantes.

In [1]:
import Pkg 


#=
Pkg.activate("new_env")
Pkg.Registry.rm("General")
Pkg.Registry.add("General")
Pkg.activate(".")
=#


Pkg.add("LinearAlgebra")
Pkg.add("ImageIO")
Pkg.add("Images")
Pkg.add("MultivariateStats")

import Pkg; Pkg.offline(true)


using LinearAlgebra, ImageIO, Images, MultivariateStats

   Resolving package versions...
    Updating `C:\Users\USER\.julia\environments\v1.11\Project.toml`
  [37e2e46d] + LinearAlgebra v1.11.0
  No Changes to `C:\Users\USER\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
   Installed libsixel_jll ─ v1.10.4+0
   Installed Giflib_jll ─── v5.2.3+0
   Installed OffsetArrays ─ v1.15.0
   Installed TiffImages ─── v0.11.2
    Updating `C:\Users\USER\.julia\environments\v1.11\Project.toml`
  [82e4d734] + ImageIO v0.6.9
    Updating `C:\Users\USER\.julia\environments\v1.11\Manifest.toml`
  [39de3d68] + AxisArrays v0.4.7
  [2803e5a7] + ImageAxes v0.6.12
  [c817782e] + ImageBase v0.1.7
  [a09fc81d] + ImageCore v0.10.5
  [82e4d734] + ImageIO v0.6.9
  [bc367c6b] + ImageMetadata v0.9.10
  [9b13fd28] + IndirectArrays v1.0.0
  [8197267c] + IntervalSets v0.7.10
  [b835a17e] + JpegTurbo v0.1.5
  [8cdb02fc] + LazyModules v0.3.1
  [e94cdb99] + MosaicViews v0.3.4
  [f09324ee] + Netpbm v1.1.1
  [6fe1bfb0] + OffsetArrays v1.15.0
  [52e1

En este paso definimos tres funciones **SVD_Maker**, **PCA_Maker** e **Image_Compressor** tal que **Image_Compressor** carga una imagen y la convierte en una matriz, ya que una imagen **RGB** esta compuesta por una matriz de $n \times d \times 3$, la función, toma la imagen y la procesa como una imagen en blanco y negro (bn) (ya que estas solo tienen una dimensión de color con valores de 0 a 255).<br><br>

Despues, se guarda la imagen en bn como comparación y se llama a las funciones **SVD_Maker** y **PCA_Maker**. Cada una toma $k=$  $5$, $50$ y $100$ tal que son los parámetros de valores singulares (**SVD**) y eigenvalores (**PCA**) que tomaremos para comprimir nuestras imagenes.<br><br>

**Sobre la nomenclatura:**<br><br>
Nuestros resultados se guardarán como archivos con nombre **Aprox_PCA_k_foto_n.jpg** y **Aprox_SVD_k_foto_n.jpg** donde $k$ es el número de eigenvectores o valores singulares que se están tomando para cada imagen y $n$ es el identificador de la foto. Ademas se incluyen imagenes **FullGrey_foto_n.jpg** que simplemente son la proyección de las imagenes originales a blanco y negro para comparar la calidad de la original con las procesadas de manera objetiva sin considerar el color.

In [16]:

# Funcion para obtener el SVD de las imagenes
# Pasamos de argumentos una matriz y el nombre del archivo 

function SVD_Maker(mat::Matrix, Image_name::String)
    
    dir_src = @__DIR__
    dir_base = joinpath(dir_src, "..")
    dir_fig = joinpath(dir_base, "fig")
    
    # Usamos la funcion svd() de la biblioteca LinearAlgebra para obtener un objeto SVD de una matriz
    F = svd(mat)
    
    # Definimos una K para tomar un numero de valores principales
    k = 5
    # Recomponemos la matriz X como un USV tal que solo tomamos K componentes 
    M = F.U[:, 1:k] * Diagonal(F.S[1:k]) * F.Vt[1:k, :]
    # Nos aseguramos que no tengamos valores negativos (ya que no tendria sentido dado que es una foto)
    M = abs.(M.*0.99)
    println("M info: Type = $(typeof(M)), size = $(size(M))")
    # Guardamos la imagen 
    save(joinpath(dir_fig,"Aprox_SVD_5_" * Image_name * ".jpg"), M)

    # Repetimos para k = 50 y k = 100
    k = 50
    M = F.U[:, 1:k] * Diagonal(F.S[1:k]) * F.Vt[1:k, :]
    M = abs.(M.*0.99)
    println("M info: Type = $(typeof(M)), size = $(size(M))")
    save(joinpath(dir_fig,"Aprox_SVD_50_" * Image_name * ".jpg"), M)

    k = 100
    M = F.U[:, 1:k] * Diagonal(F.S[1:k]) * F.Vt[1:k, :]
    M = abs.(M.*0.99)
    println("M info: Type = $(typeof(M)), size = $(size(M))")
    save(joinpath(dir_fig,"Aprox_SVD_100_" * Image_name * ".jpg"), M)

end


function PCA_Maker(mat::Matrix, Image_name::String)
    
    dir_src = @__DIR__
    dir_base = joinpath(dir_src, "..")
    dir_fig = joinpath(dir_base, "fig")

    # Volvemos a definir valores de K
    k = 5
    # Ajustamos una matriz como metodo de PCA con la funcion fit de la biblioteca
    # MultivariateStats con un numero maximo de valores principales k 
    M = fit(PCA, mat; maxoutdim = k)
    # Transformamos y recountruimos la matriz
    Y = transform(M,mat)
    mat_rec = reconstruct(M,Y)
    # Guarmos la imagen
    save(joinpath(dir_fig,"Aprox_PCA_5_" * Image_name * ".jpg"), mat_rec)

    # Repetimos para k = 50 y k = 100
    k = 50
    M = fit(PCA, mat; maxoutdim = k)
    Y = transform(M,mat)
    mat_rec = reconstruct(M,Y)
    save(joinpath(dir_fig,"Aprox_PCA_50_" * Image_name * ".jpg"), mat_rec)


    k = 100
    M = fit(PCA, mat; maxoutdim = k)
    Y = transform(M,mat)
    mat_rec = reconstruct(M,Y)
    save(joinpath(dir_fig,"Aprox_PCA_100_" * Image_name * ".jpg"), mat_rec)


end

function Image_compressor(Image_name::String)
    dir_src = @__DIR__
    dir_base = joinpath(dir_src, "..")
    dir_path = joinpath(dir_base, "dat")
    file = joinpath(dir_path, Image_name)
    image = load(file)
    # Tomamos la imagen y la volvemos a escala de grises 
    gray = Gray.(image)
    # Creamos una matriz con la imagen en grises
    mat = convert(Matrix{Float64}, gray)
        print(size(mat))
    # Guardamos la imagen en gris como referencia de la imagen original
  
    dir_fig = joinpath(dir_base, "fig")
    save(joinpath(dir_fig,"FullGrey_" * Image_name * ".jpg"), gray)

    PCA_Maker(mat,Image_name)
    SVD_Maker(mat,Image_name)


end




Image_compressor (generic function with 1 method)

## Llamada a la función

Usamos una busqueda de coincidencias para encontrar todas la imagenes que vamos a procesar y ejecutamos un for para cada imagen en las coincidencias.

In [17]:
# Abrimos el directorio donde estan las imagenes a procesar
dir_src = @__DIR__
dir_base = joinpath(dir_src, "..")
dir_path = joinpath(dir_base, "dat")
# Definimos una variable matches que contiene las coincidencias de archivos que empiezan con "foto_"
matches = filter(x -> startswith(x, "foto_"), readdir(dir_path))

println(matches)

# Para cada imagen en matches usamos la funcion Image_compressor 
for foto in matches
    println("Procesanso archivo $foto")
    Image_compressor(foto)
end



["foto_1.jpg", "foto_2.jpg", "foto_3.jpg", "foto_4.jpg", "foto_5.jpg", "foto_6.jpg"]
Procesanso archivo foto_1.jpg
(1280, 960)M info: Type = Matrix{Float64}, size = (1280, 960)
M info: Type = Matrix{Float64}, size = (1280, 960)
M info: Type = Matrix{Float64}, size = (1280, 960)
Procesanso archivo foto_2.jpg


┌ Warning: Premature end of JPEG byte sequence.
└ @ JpegTurbo C:\Users\USER\.julia\packages\JpegTurbo\HKywP\src\decode.jl:220
┌ Warning: Premature end of JPEG byte sequence.
└ @ JpegTurbo C:\Users\USER\.julia\packages\JpegTurbo\HKywP\src\decode.jl:220


(2252, 4000)M info: Type = Matrix{Float64}, size = (2252, 4000)
M info: Type = Matrix{Float64}, size = (2252, 4000)
M info: Type = Matrix{Float64}, size = (2252, 4000)
Procesanso archivo foto_3.jpg


┌ Warning: Premature end of JPEG byte sequence.
└ @ JpegTurbo C:\Users\USER\.julia\packages\JpegTurbo\HKywP\src\decode.jl:220
┌ Warning: Premature end of JPEG byte sequence.
└ @ JpegTurbo C:\Users\USER\.julia\packages\JpegTurbo\HKywP\src\decode.jl:220


(2252, 4000)M info: Type = Matrix{Float64}, size = (2252, 4000)
M info: Type = Matrix{Float64}, size = (2252, 4000)
M info: Type = Matrix{Float64}, size = (2252, 4000)
Procesanso archivo foto_4.jpg


...

(2252, 4000)M info: Type = Matrix{Float64}, size = (2252, 4000)
M info: Type = Matrix{Float64}, size = (2252, 4000)
M info: Type = Matrix{Float64}, size = (2252, 4000)
Procesanso archivo foto_5.jpg


┌ Warning: Premature end of JPEG byte sequence.
└ @ JpegTurbo C:\Users\USER\.julia\packages\JpegTurbo\HKywP\src\decode.jl:220
┌ Warning: Premature end of JPEG byte sequence.
└ @ JpegTurbo C:\Users\USER\.julia\packages\JpegTurbo\HKywP\src\decode.jl:220


(6752, 12000)M info: Type = Matrix{Float64}, size = (6752, 12000)
M info: Type = Matrix{Float64}, size = (6752, 12000)
M info: Type = Matrix{Float64}, size = (6752, 12000)
Procesanso archivo foto_6.jpg


┌ Warning: Premature end of JPEG byte sequence.
└ @ JpegTurbo C:\Users\USER\.julia\packages\JpegTurbo\HKywP\src\decode.jl:220
┌ Warning: Premature end of JPEG byte sequence.
└ @ JpegTurbo C:\Users\USER\.julia\packages\JpegTurbo\HKywP\src\decode.jl:220


(6752, 12000)M info: Type = Matrix{Float64}, size = (6752, 12000)
M info: Type = Matrix{Float64}, size = (6752, 12000)
M info: Type = Matrix{Float64}, size = (6752, 12000)


## Conclusiones: 

1. Concentración de la información en las principales componentes

En ambos métodos, se observa que a medida que $k$ disminuye, la reducción de la dimensionalidad concentra la información en las zonas de la imagen donde los detalles y los cambios son más relevantes. Este comportamiento está relacionado con el hecho de que los componentes principales se ordenan de acuerdo a la varianza explicada, y los primeros componentes suelen capturar los aspectos de mayor contraste o cambio en la imagen.

2. Pérdida de detalles en áreas homogéneas

Las zonas con menor variabilidad (por ejemplo, fondos planos o texturas uniformes) tienden a desaparecer o suavizarse conforme $k$ disminuye. Esto ocurre porque las regiones homogéneas no contribuyen significativamente a la varianza, por lo que se descartan en las primeras reducciones. Por ejemplo, en una imagen con fondo blanco, la textura sutil del fondo desaparece al reducir $k$ lo que se traduce en un fondo completamente liso.


3. Preservación de detalles cercanos al objeto central

Para valores pequeños de $k$ (como $k = 5$) los detalles se concentran en los bordes o contornos del objeto principal en la imagen. Esto se debe a que los bordes tienen altos gradientes de intensidad (es decir, cambios bruscos de color o tonalidad), y estas transiciones son capturadas por los primeros componentes principales. En términos de la percepción visual, esto implica que el sistema da prioridad a las diferencias locales significativas, que son las más perceptibles para el ojo humano.

6. Compromiso entre compresión y preservación de la información

La reducción de dimensionalidad implica un compromiso entre la compresión de la información y la preservación de los detalles.
Para valores altos de $k$ la imagen se aproxima mejor a la original, mientras que para valores bajos de $k$ se obtiene una gran compresión, pero con pérdida de detalles en regiones de baja varianza.
